# U-Net for Road Segmentation

## Required Folder Structure

The working folder contains the following subfolders:
* `training` Contains:
    * `groundtruth` Contains the groudtruth images (format: `satImage_xxx.png`)
    * `images` Contains the images to segment (format: `satImage_xxx.png`)
* `predictions_submission` Will contain the images predicted to create a submission
* `test_set_images` Contains the test images used to create a submission (format: `test_x.png` or `test_xx.png`)

This folder also contains 2 files:
* `road_segmentation_model.h5` Saved version the model. Allows to save the weights of the previous training that was done
* `smooth_tiled_prediction.py` Present in the GitHub repo.

In another folder we will store the autogenerated images. This folder should be called `augmented_set`.
In this folder there must be 2 subfolders:
* `groundtruth` contains autogenerated groundtruths
* `images` contains autogenerates images

## Pipeline

### Imports

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import argparse
import datetime
import os
import random
import re
import sys

from itertools import chain
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from PIL import Image
from skimage.io import imread, imshow
from skimage.transform import resize
import sklearn.metrics
from smooth_tiled_predictions import predict_img_with_smooth_windowing
from time import strftime
from tqdm import tqdm

Using TensorFlow backend.


### Constant Definitions

#### Non-Tweakable

In [2]:
# Image definition
IMG_WIDTH    = 400
IMG_HEIGHT   = 400
IMG_CHANNELS = 3
PIXEL_DEPTH  = 255

# Folder definitions
IMAGE_DATA_PATH           = 'training/images/'
MASK_DATA_PATH            = 'training/groundtruth/'
MODEL_SAVE_LOCATION       = 'road_segmentation_model.h5'
SUBMISSION_DATA_DIR       = 'test_set_images/'
PREDICTION_SUBMISSION_DIR = 'predictions_submission/'

IMAGES_FILENAMES = os.listdir(IMAGE_DATA_PATH)

# Image generation
OUTPUT_DATA_IMAGE_PATH = 'augmented_set/'

# Seeding
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed = SEED
tf.random.set_seed(SEED)

# Checkpoints
checkpoint_path = 'checkpoints/cp.ckpt'
checkpoint_dir  = os.path.dirname(checkpoint_path)

#### Tweakable

In [3]:
# Image generation
GENERATE_NEW_IMG   = False
USE_GENERATED_IMG  = True
IMG_TO_GEN_PER_IMG = 100

# Load existing model
USE_SAVED_MODEL = False

# F1-score estimation
NUMBER_OF_IMG_TO_TEST = 10

# Predictions
RUN_PREDICTIONS_ON_TEST_IMAGES = True

### Image Generation

In [4]:
if GENERATE_NEW_IMG:
    # load the input image, convert it to a NumPy array, and then
    # reshape it to have an extra dimension
    for img in tqdm(IMAGES_FILENAMES):
        image = load_img(IMAGE_DATA_PATH+img)
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)

        truth = load_img(MASK_DATA_PATH+img)
        truth = img_to_array(truth)
        truth = np.expand_dims(truth, axis=0)
        # construct the image generator for data augmentation then
        # initialize the total number of images generated thus far
        aug = ImageDataGenerator(rotation_range=360,
                  zoom_range=0.3,
                  brightness_range=[0.7,1],
                  width_shift_range=0.1,
                  height_shift_range=0.1,
                  vertical_flip=True,
                  shear_range=0.15,
                  horizontal_flip=True,
                  fill_mode="reflect")
        total = 0

        # construct the actual Python generator
        imageGen = aug.flow(image, y=truth, batch_size=1, save_to_dir=OUTPUT_DATA_IMAGE_PATH + "images",
          save_prefix=img.split(".")[0], save_format="png", seed = SEED )
        truthGen = aug.flow(truth, y=truth, batch_size=1, save_to_dir=OUTPUT_DATA_IMAGE_PATH + "groundtruth",
          save_prefix=img.split(".")[0], save_format="png", seed = SEED )
        # loop over examples from our image data augmentation generator
        for image in imageGen:
            # increment our counter
            total += 1

            # if we have reached the specified number of examples, break
            # from the loop
            if total == IMG_TO_GEN_PER_IMG:
                break

        total = 0
        for image in truthGen:
            # increment our counter
            total += 1

            # if we have reached the specified number of examples, break
            # from the loop
            if total == IMG_TO_GEN_PER_IMG:
                break

if(USE_GENERATED_IMG):
    print("[INFO]: Updating images_filename")
    IMAGE_DATA_PATH = OUTPUT_DATA_IMAGE_PATH+'images/'
    MASK_DATA_PATH = OUTPUT_DATA_IMAGE_PATH+ 'groundtruth/'
    print("[INFO]: new MASK_DATA_PATH : "+ MASK_DATA_PATH)
    print("[INFO]: new IMAGE_DATA_PATH : " + IMAGE_DATA_PATH)
    IMAGES_FILENAMES = os.listdir(IMAGE_DATA_PATH)
    print("[INFO]: There are " + str(len(IMAGES_FILENAMES)) + " found")

[INFO]: Updating images_filename
[INFO]: new MASK_DATA_PATH : augmented_set/groundtruth/
[INFO]: new IMAGE_DATA_PATH : augmented_set/images/
[INFO]: There are 10000 found


### Image Loading

In [5]:
np.random.seed = SEED
print("[INFO]: Loading images into RAM", flush = True)
X = np.zeros((len(IMAGES_FILENAMES), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y = np.zeros((len(IMAGES_FILENAMES), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

for n, filename in tqdm(enumerate(IMAGES_FILENAMES), total=len(IMAGES_FILENAMES)):   
    img = imread(IMAGE_DATA_PATH + filename)[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X[n] = img
    mask = imread(MASK_DATA_PATH + filename)
    mask = np.expand_dims(resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant',
                                      preserve_range=True), axis=-1)
    if USE_GENERATED_IMG:
        Y[n] = mask[:,:,0]
    else:
        Y[n] = mask

x_train=X 
y_train=Y

[INFO]: Loading images into RAM


100%|██████████| 10000/10000 [03:54<00:00, 42.58it/s]


### Model Building

In [5]:
models = []
for i in range(10):
    inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH,+ IMG_CHANNELS))
    s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(s)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(c5)

    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.relu, kernel_initializer='he_normal',
                                padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    models.append(model)

### Saved Model Loading

In [6]:
if USE_SAVED_MODEL:
    for m, model in enumerate(models):
        path = f'road_segmentation_model_{m}.h5'
        if not os.path.isfile(path):
            print("[ERROR]: Could not locate file for model weights. Proceding without loading weights.")
        else:
            model.load_weights(path)
            print("[INFO]: Loading saved model weights")

### Checkpoints for Training

In [8]:
%load_ext tensorboard
#!rm -rf ./logs/ 

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)
callbacks = [
  #tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  tensorboard_callback,
  cp_callback 
]
%tensorboard --logdir logs/fit

### Training

In [9]:
shuffled_indices = np.random.permutation(len(x_train))
x_rand = x_train[shuffled_indices]
y_rand = y_train[shuffled_indices]

In [10]:
%%time
for m, model in enumerate(models):
    model.fit(x_rand[m*1000:(m+1)*1000], y_rand[m*1000:(m+1)*1000], validation_split=0.1,
              batch_size=32, epochs=200, callbacks=callbacks, shuffle=True)
    model.save_weights(f'road_segmentation_model_{m}.h5', overwrite=True)

Train on 900 samples, validate on 100 samples
Epoch 1/200
896/900 [============================>.] - ETA: 0s - loss: 0.5903 - accuracy: 0.7209
Epoch 00001: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 16s 18ms/sample - loss: 0.5901 - accuracy: 0.7211 - val_loss: 0.5594 - val_accuracy: 0.7594
Epoch 2/200
896/900 [============================>.] - ETA: 0s - loss: 0.5325 - accuracy: 0.7630
Epoch 00002: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.5323 - accuracy: 0.7630 - val_loss: 0.5347 - val_accuracy: 0.7594
Epoch 3/200
896/900 [============================>.] - ETA: 0s - loss: 0.5079 - accuracy: 0.7627
Epoch 00003: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.5073 - accuracy: 0.7630 - val_loss: 0.5055 - val_accuracy: 0.7594
Epoch 4/200
896/900 [============================>.] - ETA: 0s - loss: 0.4965 - accuracy: 0.7632
Epoch 00004: s

Epoch 30/200
896/900 [============================>.] - ETA: 0s - loss: 0.1391 - accuracy: 0.9420
Epoch 00030: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 12s 13ms/sample - loss: 0.1389 - accuracy: 0.9421 - val_loss: 0.2291 - val_accuracy: 0.9159
Epoch 31/200
896/900 [============================>.] - ETA: 0s - loss: 0.1307 - accuracy: 0.9459
Epoch 00031: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 11s 12ms/sample - loss: 0.1309 - accuracy: 0.9458 - val_loss: 0.2485 - val_accuracy: 0.9109
Epoch 32/200
896/900 [============================>.] - ETA: 0s - loss: 0.1316 - accuracy: 0.9452
Epoch 00032: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.1317 - accuracy: 0.9451 - val_loss: 0.2021 - val_accuracy: 0.9151
Epoch 33/200
896/900 [============================>.] - ETA: 0s - loss: 0.1356 - accuracy: 0.9437
Epoch 00033: saving model to checkpoints/cp.ckpt
900/900

896/900 [============================>.] - ETA: 0s - loss: 0.0519 - accuracy: 0.9776
Epoch 00088: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0518 - accuracy: 0.9777 - val_loss: 0.2256 - val_accuracy: 0.9380
Epoch 89/200
896/900 [============================>.] - ETA: 0s - loss: 0.0520 - accuracy: 0.9776
Epoch 00089: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0520 - accuracy: 0.9776 - val_loss: 0.2432 - val_accuracy: 0.9375
Epoch 90/200
896/900 [============================>.] - ETA: 0s - loss: 0.0512 - accuracy: 0.9779
Epoch 00090: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 12ms/sample - loss: 0.0512 - accuracy: 0.9779 - val_loss: 0.2266 - val_accuracy: 0.9381
Epoch 91/200
896/900 [============================>.] - ETA: 0s - loss: 0.0532 - accuracy: 0.9772
Epoch 00091: saving model to checkpoints/cp.ckpt
900/900 [===========

896/900 [============================>.] - ETA: 0s - loss: 0.0721 - accuracy: 0.9699
Epoch 00117: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0722 - accuracy: 0.9699 - val_loss: 0.2539 - val_accuracy: 0.9218
Epoch 118/200
896/900 [============================>.] - ETA: 0s - loss: 0.0637 - accuracy: 0.9732
Epoch 00118: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 12ms/sample - loss: 0.0637 - accuracy: 0.9732 - val_loss: 0.1880 - val_accuracy: 0.9377
Epoch 119/200
896/900 [============================>.] - ETA: 0s - loss: 0.0527 - accuracy: 0.9774
Epoch 00119: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0527 - accuracy: 0.9774 - val_loss: 0.2106 - val_accuracy: 0.9394
Epoch 120/200
896/900 [============================>.] - ETA: 0s - loss: 0.0477 - accuracy: 0.9794
Epoch 00120: saving model to checkpoints/cp.ckpt
900/900 [========

896/900 [============================>.] - ETA: 0s - loss: 0.0383 - accuracy: 0.9833
Epoch 00146: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 12ms/sample - loss: 0.0383 - accuracy: 0.9833 - val_loss: 0.2505 - val_accuracy: 0.9423
Epoch 147/200
896/900 [============================>.] - ETA: 0s - loss: 0.0385 - accuracy: 0.9832
Epoch 00147: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0385 - accuracy: 0.9833 - val_loss: 0.2544 - val_accuracy: 0.9428
Epoch 148/200
896/900 [============================>.] - ETA: 0s - loss: 0.0379 - accuracy: 0.9835
Epoch 00148: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0379 - accuracy: 0.9835 - val_loss: 0.2808 - val_accuracy: 0.9403
Epoch 149/200
896/900 [============================>.] - ETA: 0s - loss: 0.0379 - accuracy: 0.9835
Epoch 00149: saving model to checkpoints/cp.ckpt
900/900 [========

...

Epoch 28/200
896/900 [============================>.] - ETA: 0s - loss: 0.1085 - accuracy: 0.9545
Epoch 00028: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 11s 12ms/sample - loss: 0.1087 - accuracy: 0.9544 - val_loss: 0.1681 - val_accuracy: 0.9315
Epoch 29/200
896/900 [============================>.] - ETA: 0s - loss: 0.1051 - accuracy: 0.9560
Epoch 00029: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.1051 - accuracy: 0.9560 - val_loss: 0.1638 - val_accuracy: 0.9336
Epoch 30/200
896/900 [============================>.] - ETA: 0s - loss: 0.1013 - accuracy: 0.9575
Epoch 00030: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.1014 - accuracy: 0.9575 - val_loss: 0.1624 - val_accuracy: 0.9338
Epoch 31/200
896/900 [============================>.] - ETA: 0s - loss: 0.1004 - accuracy: 0.9579
Epoch 00031: saving model to checkpoints/cp.ckpt
900/900

896/900 [============================>.] - ETA: 0s - loss: 0.0476 - accuracy: 0.9794
Epoch 00086: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0476 - accuracy: 0.9794 - val_loss: 0.2238 - val_accuracy: 0.9435
Epoch 87/200
896/900 [============================>.] - ETA: 0s - loss: 0.0474 - accuracy: 0.9795
Epoch 00087: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0474 - accuracy: 0.9795 - val_loss: 0.2295 - val_accuracy: 0.9439
Epoch 88/200
896/900 [============================>.] - ETA: 0s - loss: 0.0467 - accuracy: 0.9798
Epoch 00088: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 12ms/sample - loss: 0.0468 - accuracy: 0.9797 - val_loss: 0.2319 - val_accuracy: 0.9432
Epoch 89/200
896/900 [============================>.] - ETA: 0s - loss: 0.0465 - accuracy: 0.9799
Epoch 00089: saving model to checkpoints/cp.ckpt
900/900 [===========

896/900 [============================>.] - ETA: 0s - loss: 0.0419 - accuracy: 0.9818
Epoch 00115: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 12ms/sample - loss: 0.0419 - accuracy: 0.9818 - val_loss: 0.2452 - val_accuracy: 0.9461
Epoch 116/200
896/900 [============================>.] - ETA: 0s - loss: 0.0408 - accuracy: 0.9823
Epoch 00116: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0408 - accuracy: 0.9823 - val_loss: 0.2391 - val_accuracy: 0.9448
Epoch 117/200
896/900 [============================>.] - ETA: 0s - loss: 0.0405 - accuracy: 0.9824
Epoch 00117: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0405 - accuracy: 0.9824 - val_loss: 0.2456 - val_accuracy: 0.9457
Epoch 118/200
896/900 [============================>.] - ETA: 0s - loss: 0.0409 - accuracy: 0.9822
Epoch 00118: saving model to checkpoints/cp.ckpt
900/900 [========

Epoch 144/200
896/900 [============================>.] - ETA: 0s - loss: 0.0369 - accuracy: 0.9839
Epoch 00144: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 12ms/sample - loss: 0.0368 - accuracy: 0.9839 - val_loss: 0.2469 - val_accuracy: 0.9473
Epoch 145/200
896/900 [============================>.] - ETA: 0s - loss: 0.0366 - accuracy: 0.9840
Epoch 00145: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 12ms/sample - loss: 0.0366 - accuracy: 0.9840 - val_loss: 0.2768 - val_accuracy: 0.9466
Epoch 146/200
896/900 [============================>.] - ETA: 0s - loss: 0.0361 - accuracy: 0.9842
Epoch 00146: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0361 - accuracy: 0.9842 - val_loss: 0.2756 - val_accuracy: 0.9482
Epoch 147/200
896/900 [============================>.] - ETA: 0s - loss: 0.0362 - accuracy: 0.9842
Epoch 00147: saving model to checkpoints/cp.ckpt
900

Epoch 173/200
896/900 [============================>.] - ETA: 0s - loss: 0.0452 - accuracy: 0.9806
Epoch 00173: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 11s 12ms/sample - loss: 0.0452 - accuracy: 0.9806 - val_loss: 0.2545 - val_accuracy: 0.9426
Epoch 174/200
896/900 [============================>.] - ETA: 0s - loss: 0.0438 - accuracy: 0.9811
Epoch 00174: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0439 - accuracy: 0.9811 - val_loss: 0.2543 - val_accuracy: 0.9441
Epoch 175/200
896/900 [============================>.] - ETA: 0s - loss: 0.0433 - accuracy: 0.9814
Epoch 00175: saving model to checkpoints/cp.ckpt
900/900 [==============================] - 10s 11ms/sample - loss: 0.0432 - accuracy: 0.9814 - val_loss: 0.2696 - val_accuracy: 0.9444
Epoch 176/200
896/900 [============================>.] - ETA: 0s - loss: 0.0423 - accuracy: 0.9817
Epoch 00176: saving model to checkpoints/cp.ckpt
900

### F1-Score Estimation

In [ ]:
 #https://github.com/kiteco/kite-python-blog-post-code/blob/master/image-segmentation/image_segmentation.py

"""
Helper functions used to estimate F1 score.
"""

def get_f1_score(groundtruth_list, predicted_list):
    """Return f1 score covering edge cases"""

    tn, fp, fn, tp = get_confusion_matrix_elements(groundtruth_list, predicted_list)
    
    if _all_class_0_predicted_as_class_0(groundtruth_list, predicted_list) is True:
        f1_score = 1
    elif _all_class_1_predicted_as_class_1(groundtruth_list, predicted_list) is True:
        f1_score = 1
    else:
        f1_score = (2 * tp) / ((2 * tp) + fp + fn)

    return f1_score

def get_confusion_matrix_elements(groundtruth_list, predicted_list):
    """returns confusion matrix elements i.e TN, FP, FN, TP as floats
	See example code for helper function definitions
    """
    _assert_valid_lists(groundtruth_list, predicted_list)

    if _all_class_1_predicted_as_class_1(groundtruth_list, predicted_list) is True:
        tn, fp, fn, tp = 0, 0, 0, np.float64(len(groundtruth_list))

    elif _all_class_0_predicted_as_class_0(groundtruth_list, predicted_list) is True:
        tn, fp, fn, tp = np.float64(len(groundtruth_list)), 0, 0, 0

    else:
        tn, fp, fn, tp = sklearn.metrics.confusion_matrix(groundtruth_list, predicted_list).ravel()
        tn, fp, fn, tp = np.float64(tn), np.float64(fp), np.float64(fn), np.float64(tp)

    return tn, fp, fn, tp

def _assert_valid_lists(groundtruth_list, predicted_list):
    assert len(groundtruth_list) == len(predicted_list)
    for unique_element in np.unique(groundtruth_list).tolist():
        assert unique_element in [0, 1]

def _all_class_1_predicted_as_class_1(groundtruth_list, predicted_list):
    _assert_valid_lists(groundtruth_list, predicted_list)
    return np.unique(groundtruth_list).tolist() == np.unique(predicted_list).tolist() == [1]


def _all_class_0_predicted_as_class_0(groundtruth_list, predicted_list):
    _assert_valid_lists(groundtruth_list, predicted_list)
    return np.unique(groundtruth_list).tolist() == np.unique(predicted_list).tolist() == [0]

print("[INFO]: Estimating F1 score", flush=True)
step = 0.001
start = 0.51
stop = 0.53
F1s = []
thresholds = np.arange(start,stop+step,step)
for threshold in thresholds:
    F1_accumulator = 0

    for idx in tqdm(range(NUMBER_OF_IMG_TO_TEST)):
        x=np.array(x_train[idx])
        x=np.expand_dims(x, axis=0)
        predict = model.predict(x, verbose=0)

        predict = (predict > threshold).astype(np.uint8)
        f1 = get_f1_score(y_train[idx].flatten(),predict[0].flatten())
        F1_accumulator += f1
    #print("\nAverage F1: " + str(F1_accumulator / NUMBER_OF_IMG_TO_TEST))
    F1s.append(F1_accumulator / NUMBER_OF_IMG_TO_TEST)
best_index = np.argmax(F1s)
best_threshold = thresholds[best_index]
best_F1 = F1s[best_index]
print(f"\nThe best threshold is {best_threshold}, yielding an average F1 score of {best_F1*100:.2f}")
plt.plot(thresholds, F1s);

### Image Demo on Training Set

In [ ]:
def get_prediction(img):
    x=np.array(img)
    x=np.expand_dims(x, axis=0)
    predict = model.predict(x, verbose=1)
    print(predict.shape)
    predict = (predict > best_threshold).astype(np.uint8)
    predict = np.squeeze(predict[0])
    print(predict.shape)

    return predict

idx = 4
prediction = get_prediction(x_train[idx])

print("- Original Image:")
imshow(x_train[idx])
plt.show()

print("- Prediction:")
imshow(prediction)
plt.show()

print("- Original Groundtruth:")
imshow(np.squeeze(y_train[idx]));

### Running Predictions on Test Set

In [ ]:
def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * PIXEL_DEPTH).round().astype(np.uint8)
    return rimg

if RUN_PREDICTIONS_ON_TEST_IMAGES:
    print("[INFO]: Running prediction on submission set")
    if not os.path.isdir(PREDICTION_SUBMISSION_DIR):
        os.mkdir(PREDICTION_SUBMISSION_DIR)
    predictions_per_model = []
    for model in models:
        predictions = []
        for i in range(1, 51):
            pimg = imread(SUBMISSION_DATA_DIR + f"test_{i}.png")[:,:,:IMG_CHANNELS]
            predictions.append(predict_img_with_smooth_windowing(
                pimg,
                window_size=IMG_WIDTH,
                subdivisions=2,  # Minimal amount of overlap for windowing. Must be an even number.
                nb_classes=1,
                pred_func=(
                    lambda img_batch_subdiv: model.predict(img_batch_subdiv)
                )
                )
            )
        predictions_per_model.append(predictions)
else:
    print("[INFO]: Skipping predicting test images")

  0%|          | 0/8 [00:00<?, ?it/s]

[INFO]: Running prediction on submission set


100%|██████████| 8/8 [00:17<00:00,  2.20s/it]


100%|██████████| 8/8 [00:30<00:00,  3.83s/it]


  0%|          | 0/8 [00:00<?, ?it/s]

### Prediction Saving

In [ ]:
best_threshold = 0.5 # Default
if RUN_PREDICTIONS_ON_TEST_IMAGES:
    print("[INFO]: Writing prediction to drive")

    ROAD_THRESHOLD = 0.25#0.1

    for p, predictions in enumerate(predictions_per_model):
        pred = np.array(predictions.copy())
        for i in range(1, 51):
            pimg = pred[i-1]
            w = pimg.shape[0]
            h = pimg.shape[1]
            cimg = np.zeros((w, h, 3), dtype=np.uint8)
            pimg = (pimg > best_threshold).astype(np.uint8)
            pimg8 = np.squeeze(img_float_to_uint8(pimg))
            cimg[:, :, 0] = pimg8
            cimg[:, :, 1] = pimg8
            cimg[:, :, 2] = pimg8
            Image.fromarray(cimg).save(PREDICTION_SUBMISSION_DIR + f"gt_{p}_{i}.png")
else:
    print("[INFO]: Skipping write of predictions to disk")

### AICrowd Submission Creation

In [ ]:
# Creating ouput for submission

foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0

def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))

def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))


time = strftime("%Y%m%dT%H%M%S")
submission_filename = f'submission-{time}.csv'
image_filenames = []
for i in range(1, 51):
    image_filename = f'{PREDICTION_SUBMISSION_DIR}gt_{i}.png'
    image_filenames.append(image_filename)
masks_to_submission(submission_filename, *image_filenames)

## Hyperparameter Tweaking

### Prediction Threshold

In [ ]:
i = 23
input_img = imread(SUBMISSION_DATA_DIR + f"test_{i}.png")[:,:,:IMG_CHANNELS]

predictions_smooth = predict_img_with_smooth_windowing(
    input_img,
    window_size=IMG_WIDTH,
    subdivisions=2,  # Minimal amount of overlap for windowing. Must be an even number.
    nb_classes=1,
    pred_func=(
        lambda img_batch_subdiv: model.predict(img_batch_subdiv)
    )
)

predict = (predictions_smooth > 0.25).astype(np.uint8)

imshow(np.squeeze(predict))
plt.show()
imshow(input_img)

## Tries

### 1. 07.12.2019

```
SEED = 42
IMG_TO_GEN_PER_IMG = 100
aug = ImageDataGenerator(rotation_range=360,
                  zoom_range=0.3,
                  brightness_range=[0.7,1],
                  width_shift_range=0.1,
                  height_shift_range=0.1,
                  vertical_flip=True,
                  shear_range=0.15,
                  horizontal_flip=True,
                  fill_mode="reflect")
```

i.e. 10'000 images, split between 9'000 for training and 1'000 for validation

400 epochs, weights and loss/accuracy evolution saved on Robin's home computer

Prediction thresholds (not foreground thresholds):
- 0.100 -> 0.863 F1 AICrowd
- 0.250 -> 0.881 F1 AICrowd
- 0.514 -> 0.893 F1 AICrowd

## References

- https://androidkt.com/tensorflow-keras-unet-for-image-image-segmentation/

- https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277

- https://arxiv.org/pdf/1505.04597.pdf